<a href="https://colab.research.google.com/github/bellDataSc/Engenharia-de-Dados-Processando-e-Analisando-Notas-Fiscais-GOV-BR/blob/main/Data_Engineering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineering - Google Colab Demo

**Made with ☕ by [Isabel Cruz](https://github.com/bellDataSc)**


**Demonstração prática de engenharia de dados com análise de imagens e texto**

Este notebook demonstra habilidades avançadas de engenharia de dados de forma **simples, mas sofisticada**.

## Objetivos
- Demonstrar processamento de dados multimodal
- Showcases de código limpo e profissional
- Pipeline de dados end-to-end
- Visualizações interativas e insights

## Stack Tecnológico
- **Python**: Linguagem principal
- **OpenCV**: Processamento de imagens
- **Transformers**: Análise de texto e NLP
- **Plotly**: Visualizações interativas
- **Pandas**: Manipulação de dados

---

## Setup e Configuração



Instalação das dependências e configuração do ambiente.

In [12]:

!pip install opencv-python-headless ultralytics plotly transformers torch
!pip install pandas numpy matplotlib seaborn pillow

print(" Dependências instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00
 Dependências instaladas com sucesso!


In [13]:

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from PIL import Image
import io
import json
import logging
from datetime import datetime
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')


plt.style.use('default')
sns.set_palette("husl")

print(" Bibliotecas importadas com sucesso!")
# print(f" Versão do Python: {sys.version}")
print(f" OpenCV versão: {cv2.__version__}")

 Bibliotecas importadas com sucesso!
 OpenCV versão: 4.12.0


## Processador de Imagens - Demonstração

Implementação de um processador de imagens que demonstra boas práticas de engenharia de dados.

In [14]:
class DataEngineeringImageProcessor:
    """
    Processador de imagens demonstrando habilidades de engenharia de dados
    Foco: código limpo, logging, tratamento de erros, e pipeline eficiente
    """

    def __init__(self, log_level=logging.INFO):
        """Inicializa o processador com configuração de logging"""
        self.logger = self._setup_logging(log_level)
        self.processing_stats = []
        self.cache = {}

        self.logger.info("Image Processor inicializado")

    def _setup_logging(self, level):
        """Configura logging estruturado"""
        logger = logging.getLogger('ImageProcessor')
        logger.setLevel(level)

        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)

        return logger

    def analyze_image_from_url(self, image_url: str) -> Dict[str, Any]:
        """
        Analisa imagem a partir de URL
        Demonstra: tratamento de erros, cache, e pipeline de dados
        """
        try:

            if image_url in self.cache:
                self.logger.info(f"Cache hit para: {image_url}")
                return self.cache[image_url]

            self.logger.info(f"Processando imagem: {image_url}")


            import requests
            response = requests.get(image_url, timeout=10)
            response.raise_for_status()

            image_array = np.frombuffer(response.content, np.uint8)
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

            if image is None:
                raise ValueError("Não foi possível decodificar a imagem")


            results = self._perform_analysis(image, image_url)


            self.cache[image_url] = results


            self.processing_stats.append({
                'url': image_url,
                'status': 'success',
                'timestamp': datetime.now(),
                'quality_score': results['quality_metrics']['overall_score']
            })

            self.logger.info(f"Análise concluída para: {image_url}")
            return results

        except Exception as e:
            self.logger.error(f"Erro ao processar {image_url}: {e}")


            self.processing_stats.append({
                'url': image_url,
                'status': 'error',
                'timestamp': datetime.now(),
                'error': str(e)
            })

            return {'error': str(e), 'url': image_url}

    def _perform_analysis(self, image: np.ndarray, source_url: str) -> Dict[str, Any]:
        """Executa análise completa da imagem"""
        height, width = image.shape[:2]

        return {
            'source_url': source_url,
            'dimensions': {'width': width, 'height': height},
            'total_pixels': width * height,
            'quality_metrics': self._assess_quality(image),
            'color_analysis': self._analyze_colors(image),
            'content_features': self._extract_features(image),
            'processing_timestamp': datetime.now().isoformat()
        }

    def _assess_quality(self, image: np.ndarray) -> Dict[str, float]:
        """Avalia qualidade usando métricas computacionais"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


        sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()


        brightness = np.mean(gray)
        contrast = np.std(gray)


        quality_score = min(100, (sharpness/100 + contrast/50) * 25 +
                           (100 - abs(brightness - 127.5)) * 0.4)

        return {
            'sharpness': round(sharpness, 2),
            'brightness': round(brightness, 2),
            'contrast': round(contrast, 2),
            'overall_score': round(quality_score, 2)
        }

    def _analyze_colors(self, image: np.ndarray) -> Dict[str, Any]:
        """Análise de cores dominantes"""

        small_image = cv2.resize(image, (150, 150))
        pixels = small_image.reshape(-1, 3).astype(np.float32)


        k = 3
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
        _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)


        unique, counts = np.unique(labels, return_counts=True)
        percentages = (counts / len(labels)) * 100


        dominant_colors = []
        for i, (center, percentage) in enumerate(zip(centers, percentages)):
            color_bgr = center.astype(int)
            color_rgb = [int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])]

            dominant_colors.append({
                'rank': i + 1,
                'rgb': color_rgb,
                'hex': f"#{color_rgb[0]:02x}{color_rgb[1]:02x}{color_rgb[2]:02x}",
                'percentage': round(percentage, 1)
            })

        return {
            'dominant_colors': sorted(dominant_colors, key=lambda x: x['percentage'], reverse=True),
            'color_diversity': round(np.std(percentages), 2)
        }

    def _extract_features(self, image: np.ndarray) -> Dict[str, Any]:
        """Extrai características da imagem"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.sum(edges > 0) / edges.size


        texture_score = np.var(gray)


        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist_norm = hist / hist.sum()
        entropy = -np.sum(hist_norm * np.log2(hist_norm + 1e-7))

        return {
            'edge_density': round(edge_density, 4),
            'texture_score': round(texture_score, 2),
            'histogram_entropy': round(entropy, 2)
        }

    def batch_process_urls(self, image_urls: List[str]) -> pd.DataFrame:
        """
        Processa múltiplas imagens em lote
        Demonstra: pipeline de dados, monitoramento de progresso
        """
        self.logger.info(f"Iniciando processamento em lote de {len(image_urls)} imagens")

        results = []

        for i, url in enumerate(image_urls, 1):
            result = self.analyze_image_from_url(url)


            if 'error' not in result:
                flat_result = {
                    'url': result['source_url'],
                    'width': result['dimensions']['width'],
                    'height': result['dimensions']['height'],
                    'total_pixels': result['total_pixels'],
                    'quality_score': result['quality_metrics']['overall_score'],
                    'sharpness': result['quality_metrics']['sharpness'],
                    'brightness': result['quality_metrics']['brightness'],
                    'contrast': result['quality_metrics']['contrast'],
                    'edge_density': result['content_features']['edge_density'],
                    'texture_score': result['content_features']['texture_score'],
                    'dominant_color_1': result['color_analysis']['dominant_colors'][0]['hex'],
                    'color_diversity': result['color_analysis']['color_diversity']
                }
                results.append(flat_result)


            if i % 5 == 0 or i == len(image_urls):
                self.logger.info(f"Progresso: {i}/{len(image_urls)} imagens processadas")

        df = pd.DataFrame(results)
        self.logger.info(f"Processamento em lote concluído: {len(df)} sucessos")

        return df

    def get_processing_summary(self) -> Dict[str, Any]:
        """Retorna resumo das estatísticas de processamento"""
        if not self.processing_stats:
            return {'message': 'Nenhuma estatística disponível'}

        total = len(self.processing_stats)
        successful = len([s for s in self.processing_stats if s['status'] == 'success'])

        return {
            'total_processed': total,
            'successful': successful,
            'success_rate': round((successful / total) * 100, 1) if total > 0 else 0,
            'cache_entries': len(self.cache),
            'last_processed': self.processing_stats[-1]['timestamp'].isoformat()
        }


image_processor = DataEngineeringImageProcessor()
print("Processador de Imagens inicializado e pronto para uso!")

2025-09-17 16:51:28,107 - ImageProcessor - INFO - Image Processor inicializado
INFO:ImageProcessor:Image Processor inicializado


Processador de Imagens inicializado e pronto para uso!


## Processador de Texto - Demonstração

Implementação de análise de texto com foco em boas práticas de engenharia de dados.

In [15]:
class DataEngineeringTextProcessor:
    """
    Processador de texto demonstrando engenharia de dados e NLP
    Foco: pipeline eficiente, análise robusta, tratamento de erros
    """

    def __init__(self):
        """Inicializa com configuração inteligente"""
        self.logger = self._setup_logging()
        self.processing_stats = []
        self.cache = {}


        self.sentiment_analyzer = self._load_sentiment_analyzer()


        self.stopwords = self._load_stopwords()

        self.logger.info("Text Processor inicializado")

    def _setup_logging(self):
        """Configura logging"""
        logger = logging.getLogger('TextProcessor')
        logger.setLevel(logging.INFO)
        return logger

    def _load_sentiment_analyzer(self):
        """Carrega analisador de sentimento com fallback"""
        try:
            from transformers import pipeline
            analyzer = pipeline("sentiment-analysis")
            self.logger.info("Transformers sentiment analyzer carregado")
            return analyzer
        except Exception as e:
            self.logger.warning(f"Não foi possível carregar Transformers: {e}")
            self.logger.info("Usando análise de sentimento por léxico")
            return None

    def _load_stopwords(self):
        """Carrega stopwords básicas"""
        return {
            'english': {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'},
            'portuguese': {'o', 'a', 'de', 'do', 'da', 'em', 'um', 'uma', 'para', 'com', 'não', 'que', 'se'},
            'spanish': {'el', 'la', 'de', 'en', 'y', 'que', 'es', 'se', 'no', 'te', 'lo', 'le', 'da', 'su'}
        }

    def analyze_text(self, text: str, text_id: str = None) -> Dict[str, Any]:
        """
        Análise completa de texto
        Demonstra: pipeline de NLP, cache, métricas abrangentes
        """
        try:
            if not text or not text.strip():
                raise ValueError("Texto vazio fornecido")

            text_id = text_id or f"text_{len(self.processing_stats) + 1}"


            cache_key = hash(text)
            if cache_key in self.cache:
                self.logger.info(f"Cache hit para texto {text_id}")
                return self.cache[cache_key]

            self.logger.info(f"Processando texto: {text_id}")


            cleaned_text = self._clean_text(text)

            results = {
                'text_id': text_id,
                'original_length': len(text),
                'cleaned_length': len(cleaned_text),
                'basic_stats': self._get_basic_stats(cleaned_text),
                'sentiment_analysis': self._analyze_sentiment(cleaned_text),
                'keyword_extraction': self._extract_keywords(cleaned_text),
                'language_detection': self._detect_language(cleaned_text),
                'readability_metrics': self._calculate_readability(cleaned_text),
                'processing_timestamp': datetime.now().isoformat()
            }

            # Cache e stats
            self.cache[cache_key] = results
            self.processing_stats.append({
                'text_id': text_id,
                'status': 'success',
                'word_count': results['basic_stats']['word_count'],
                'timestamp': datetime.now()
            })

            self.logger.info(f"Análise concluída para: {text_id}")
            return results

        except Exception as e:
            self.logger.error(f"Erro ao processar texto {text_id}: {e}")
            return {'error': str(e), 'text_id': text_id}

    def _clean_text(self, text: str) -> str:
        """Limpeza e normalização do texto"""
        import re


        cleaned = re.sub(r'\s+', ' ', text.strip())


        cleaned = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned)


        cleaned = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', cleaned)

        return cleaned.strip()

    def _get_basic_stats(self, text: str) -> Dict[str, int]:
        """Estatísticas básicas do texto"""
        import re

        words = text.split()
        sentences = [s for s in re.split(r'[.!?]+', text) if s.strip()]

        return {
            'character_count': len(text),
            'word_count': len(words),
            'sentence_count': len(sentences),
            'avg_word_length': round(np.mean([len(word) for word in words]) if words else 0, 2),
            'avg_sentence_length': round(len(words) / len(sentences) if sentences else 0, 2)
        }

    def _analyze_sentiment(self, text: str) -> Dict[str, Any]:
        """Análise de sentimento com múltiplas abordagens"""
        if self.sentiment_analyzer and len(text) < 512:
            try:
                result = self.sentiment_analyzer(text)[0]
                return {
                    'method': 'transformer',
                    'label': result['label'],
                    'confidence': round(result['score'], 3),
                    'positive_score': result['score'] if result['label'] == 'POSITIVE' else 1 - result['score']
                }
            except Exception:
                pass


        # Fallback: análise por léxico
        # Conferir com outros projetos que tenho sobre analise por léxico

        return self._lexicon_sentiment(text)

    def _lexicon_sentiment(self, text: str) -> Dict[str, Any]:
        """Análise de sentimento baseada em léxico"""
        positive_words = ['good', 'great', 'excellent', 'amazing', 'love', 'like', 'happy', 'perfect']
        negative_words = ['bad', 'terrible', 'awful', 'hate', 'dislike', 'angry', 'sad', 'poor']

        words = text.lower().split()
        pos_count = sum(1 for word in words if word in positive_words)
        neg_count = sum(1 for word in words if word in negative_words)

        total = pos_count + neg_count
        if total == 0:
            return {'method': 'lexicon', 'label': 'NEUTRAL', 'confidence': 0.5, 'positive_score': 0.5}

        pos_score = pos_count / total
        label = 'POSITIVE' if pos_score > 0.5 else 'NEGATIVE'
        confidence = max(pos_score, 1 - pos_score)

        return {
            'method': 'lexicon',
            'label': label,
            'confidence': round(confidence, 3),
            'positive_score': round(pos_score, 3)
        }

    def _extract_keywords(self, text: str, top_k: int = 10) -> List[Dict[str, Any]]:
        """Extração de palavras-chave usando TF"""
        import re
        from collections import Counter


        words = re.findall(r'\b[a-zA-Z]{3,}\b', text.lower())


        basic_stopwords = {'the', 'and', 'for', 'are', 'but', 'not', 'you', 'all', 'can', 'had', 'her', 'was', 'one', 'our', 'out', 'day', 'had', 'his', 'how', 'man', 'new', 'now', 'old', 'see', 'two', 'way', 'who', 'its', 'did', 'get', 'has', 'him', 'off', 'she', 'use', 'may', 'say', 'she', 'use', 'her', 'each', 'which', 'their', 'time', 'will', 'about', 'would', 'there', 'could', 'other'}
        filtered_words = [word for word in words if word not in basic_stopwords]


        word_counts = Counter(filtered_words)
        total_words = len(filtered_words)

        keywords = []
        for word, count in word_counts.most_common(top_k):
            tf_score = count / total_words if total_words > 0 else 0
            keywords.append({
                'word': word,
                'count': count,
                'tf_score': round(tf_score, 4)
            })

        return keywords

    def _detect_language(self, text: str) -> str:
        """Detecção simples de idioma"""
        import re

        text_lower = text.lower()

        # Indicadores por idioma
        pt_indicators = len(re.findall(r'\b(que|não|com|para|uma|dos|das|pela|pelo|sendo|tendo|onde|quando|muito|também|só|até|bem|nem|já|ainda|mais)\b', text_lower))
        es_indicators = len(re.findall(r'\b(que|no|con|para|una|los|las|por|ser|estar|muy|también|sólo|hasta|bien|ni|ya|aún|más)\b', text_lower))
        en_indicators = len(re.findall(r'\b(the|and|for|are|but|not|you|all|can|had|her|was|one|our|out|day|get|has|him|off|she|use|may|say|each|which|their|time|will|about|would|there|could|other)\b', text_lower))

        scores = {'portuguese': pt_indicators, 'spanish': es_indicators, 'english': en_indicators}
        detected = max(scores, key=scores.get) if max(scores.values()) > 2 else 'english'

        return detected

    def _calculate_readability(self, text: str) -> Dict[str, float]:
        """Métricas de legibilidade"""
        import re

        words = text.split()
        sentences = [s for s in re.split(r'[.!?]+', text) if s.strip()]

        if not sentences or not words:
            return {'flesch_score': 0, 'complexity': 0}

        avg_sentence_length = len(words) / len(sentences)
        avg_word_length = np.mean([len(word) for word in words])


        flesch_score = max(0, min(100, 206.835 - (1.015 * avg_sentence_length) - (84.6 * avg_word_length / 2)))


        unique_words = len(set(word.lower() for word in words))
        complexity = unique_words / len(words) if words else 0

        return {
            'flesch_score': round(flesch_score, 2),
            'complexity': round(complexity, 3),
            'avg_sentence_length': round(avg_sentence_length, 2),
            'avg_word_length': round(avg_word_length, 2)
        }

    def batch_process_texts(self, texts: List[str], text_ids: List[str] = None) -> pd.DataFrame:
        """
        Processamento em lote de textos
        Demonstra: pipeline escalável, monitoramento de progresso
        """
        self.logger.info(f"Iniciando processamento em lote de {len(texts)} textos")

        if text_ids and len(text_ids) != len(texts):
            raise ValueError("Comprimento de text_ids deve ser igual ao de texts")

        results = []

        for i, text in enumerate(texts):
            text_id = text_ids[i] if text_ids else f"text_{i+1}"
            result = self.analyze_text(text, text_id)

            # Achatar para DataFrame
            if 'error' not in result:
                flat_result = {
                    'text_id': result['text_id'],
                    'word_count': result['basic_stats']['word_count'],
                    'sentence_count': result['basic_stats']['sentence_count'],
                    'avg_word_length': result['basic_stats']['avg_word_length'],
                    'sentiment_label': result['sentiment_analysis']['label'],
                    'sentiment_confidence': result['sentiment_analysis']['confidence'],
                    'positive_score': result['sentiment_analysis']['positive_score'],
                    'detected_language': result['language_detection'],
                    'flesch_score': result['readability_metrics']['flesch_score'],
                    'complexity': result['readability_metrics']['complexity'],
                    'top_keyword': result['keyword_extraction'][0]['word'] if result['keyword_extraction'] else None
                }
                results.append(flat_result)


            if (i + 1) % 10 == 0 or (i + 1) == len(texts):
                self.logger.info(f"Progresso: {i+1}/{len(texts)} textos processados")

        df = pd.DataFrame(results)
        self.logger.info(f"Processamento em lote concluído: {len(df)} sucessos")

        return df

    def get_processing_summary(self) -> Dict[str, Any]:
        """Resumo das estatísticas de processamento"""
        if not self.processing_stats:
            return {'message': 'Nenhuma estatística disponível'}

        total = len(self.processing_stats)
        successful = len([s for s in self.processing_stats if s['status'] == 'success'])
        word_counts = [s['word_count'] for s in self.processing_stats if s['status'] == 'success']

        return {
            'total_processed': total,
            'successful': successful,
            'success_rate': round((successful / total) * 100, 1) if total > 0 else 0,
            'total_words_processed': sum(word_counts),
            'avg_words_per_text': round(np.mean(word_counts), 1) if word_counts else 0,
            'cache_entries': len(self.cache)
        }


text_processor = DataEngineeringTextProcessor()
print("Processador de Texto inicializado e pronto para uso!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
INFO:TextProcessor:Transformers sentiment analyzer carregado
INFO:TextProcessor:Text Processor inicializado


Processador de Texto inicializado e pronto para uso!


## Demonstração Prática - Pipeline de Dados

Vamos executar um pipeline completo de análise de dados demonstrando as capacidades da solução.

In [16]:

print("Executando Pipeline de Demonstração")
print("=" * 50)

print("\n 1. Análise de Imagens")
sample_image_urls = [
    "https://picsum.photos/800/600?random=1",
    "https://picsum.photos/600/400?random=2",
    "https://picsum.photos/1024/768?random=3",
    "https://picsum.photos/500/500?random=4",
    "https://picsum.photos/900/600?random=5"
]


image_results_df = image_processor.batch_process_urls(sample_image_urls)

print(f"\n Imagens processadas: {len(image_results_df)}")
print("\n Primeiros resultados:")
display(image_results_df.head())

# Estatísticas do processamento de imagens
image_stats = image_processor.get_processing_summary()
print(f"\n Estatísticas de Processamento de Imagens:")
for key, value in image_stats.items():
    print(f"   {key}: {value}")

2025-09-17 16:51:28,774 - ImageProcessor - INFO - Iniciando processamento em lote de 5 imagens
INFO:ImageProcessor:Iniciando processamento em lote de 5 imagens
2025-09-17 16:51:28,782 - ImageProcessor - INFO - Processando imagem: https://picsum.photos/800/600?random=1
INFO:ImageProcessor:Processando imagem: https://picsum.photos/800/600?random=1


Executando Pipeline de Demonstração

 1. Análise de Imagens


2025-09-17 16:51:29,229 - ImageProcessor - INFO - Análise concluída para: https://picsum.photos/800/600?random=1
INFO:ImageProcessor:Análise concluída para: https://picsum.photos/800/600?random=1
2025-09-17 16:51:29,232 - ImageProcessor - INFO - Processando imagem: https://picsum.photos/600/400?random=2
INFO:ImageProcessor:Processando imagem: https://picsum.photos/600/400?random=2
2025-09-17 16:51:29,563 - ImageProcessor - INFO - Análise concluída para: https://picsum.photos/600/400?random=2
INFO:ImageProcessor:Análise concluída para: https://picsum.photos/600/400?random=2
2025-09-17 16:51:29,566 - ImageProcessor - INFO - Processando imagem: https://picsum.photos/1024/768?random=3
INFO:ImageProcessor:Processando imagem: https://picsum.photos/1024/768?random=3
2025-09-17 16:51:29,906 - ImageProcessor - INFO - Análise concluída para: https://picsum.photos/1024/768?random=3
INFO:ImageProcessor:Análise concluída para: https://picsum.photos/1024/768?random=3
2025-09-17 16:51:29,910 - ImageP


 Imagens processadas: 5

 Primeiros resultados:


,url,width,height,total_pixels,quality_score,sharpness,brightness,contrast,edge_density,texture_score,dominant_color_1,color_diversity
0,https://picsum.photos/800/600?random=1,800,600,480000,96.87,135.65,129.22,47.30,0.0308,2237.32,#b4b7a8,0.48
1,https://picsum.photos/600/400?random=2,600,400,240000,100.00,6791.22,131.16,56.51,0.2526,3193.23,#54454e,3.78
2,https://picsum.photos/1024/768?random=3,1024,768,786432,100.00,1967.13,137.22,81.21,0.1751,6595.03,#c9deea,8.76
3,https://picsum.photos/500/500?random=4,500,500,250000,100.00,389.25,98.46,76.24,0.1596,5813.08,#250c11,5.35
4,https://picsum.photos/900/600?random=5,900,600,540000,100.00,1750.08,66.02,54.28,0.1206,2945.97,#2f2a25,32.46



 Estatísticas de Processamento de Imagens:
   total_processed: 5
   successful: 5
   success_rate: 100.0
   cache_entries: 5
   last_processed: 2025-09-17T16:51:31.411550


In [17]:

print("\n 2. Análise de Texto")

sample_texts = [
    "This is an amazing demonstration of data engineering capabilities using open source technologies. The results are fantastic and show great potential.",
    "Data engineering is crucial for modern businesses. It involves collecting, processing, and analyzing large volumes of data to extract meaningful insights.",
    "Machine learning and artificial intelligence are transforming industries worldwide. These technologies enable automation and improved decision-making.",
    "The integration of multiple data sources requires sophisticated pipeline orchestration and robust error handling mechanisms.",
    "Visualization and reporting are essential components of any data engineering solution. They provide stakeholders with actionable insights."
]


text_results_df = text_processor.batch_process_texts(sample_texts)

print(f"\n Textos processados: {len(text_results_df)}")
print("\n Primeiros resultados:")
display(text_results_df.head())

# Estatísticas do processamento de texto
text_stats = text_processor.get_processing_summary()
print(f"\n Estatísticas de Processamento de Texto:")
for key, value in text_stats.items():
    print(f"   {key}: {value}")

INFO:TextProcessor:Iniciando processamento em lote de 5 textos
INFO:TextProcessor:Processando texto: text_1



 2. Análise de Texto


INFO:TextProcessor:Análise concluída para: text_1
INFO:TextProcessor:Processando texto: text_2
INFO:TextProcessor:Análise concluída para: text_2
INFO:TextProcessor:Processando texto: text_3
INFO:TextProcessor:Análise concluída para: text_3
INFO:TextProcessor:Processando texto: text_4
INFO:TextProcessor:Análise concluída para: text_4
INFO:TextProcessor:Processando texto: text_5
INFO:TextProcessor:Análise concluída para: text_5
INFO:TextProcessor:Progresso: 5/5 textos processados
INFO:TextProcessor:Processamento em lote concluído: 5 sucessos



 Textos processados: 5

 Primeiros resultados:


,text_id,word_count,sentence_count,avg_word_length,sentiment_label,sentiment_confidence,positive_score,detected_language,flesch_score,complexity,top_keyword
0,text_1,21,2,6.14,POSITIVE,1.000,0.999886,english,0,1.000,this
1,text_2,21,2,6.38,POSITIVE,0.998,0.998479,english,0,0.952,data
2,text_3,16,2,8.44,POSITIVE,1.000,0.999605,english,0,0.938,machine
3,text_4,15,1,7.33,NEGATIVE,0.511,0.488609,english,0,1.000,integration
4,text_5,17,2,7.18,POSITIVE,0.999,0.999306,english,0,1.000,visualization



 Estatísticas de Processamento de Texto:
   total_processed: 5
   successful: 5
   success_rate: 100.0
   total_words_processed: 90
   avg_words_per_text: 18.0
   cache_entries: 5


## Visualizações Avançadas - Data Insights

Criação de visualizações sofisticadas para apresentar os insights dos dados processados.

In [18]:

print("Criando Visualizações Avançadas")
print("=" * 40)


fig_image_quality = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Qualidade', 'Nitidez vs Contraste',
                   'Brilho vs Qualidade', 'Densidade de Bordas'),
    specs=[[{"type": "histogram"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "box"}]]
)


fig_image_quality.add_trace(
    go.Histogram(x=image_results_df['quality_score'], name='Quality Score', nbinsx=10),
    row=1, col=1
)


fig_image_quality.add_trace(
    go.Scatter(x=image_results_df['sharpness'], y=image_results_df['contrast'],
              mode='markers', name='Sharpness vs Contrast',
              marker=dict(size=10, color=image_results_df['quality_score'],
                         colorscale='Viridis', showscale=True)),
    row=1, col=2
)


fig_image_quality.add_trace(
    go.Scatter(x=image_results_df['brightness'], y=image_results_df['quality_score'],
              mode='markers', name='Brightness vs Quality',
              marker=dict(size=12, color='lightblue')),
    row=2, col=1
)


fig_image_quality.add_trace(
    go.Box(y=image_results_df['edge_density'], name='Edge Density'),
    row=2, col=2
)

fig_image_quality.update_layout(
    height=800,
    title_text="Análise Avançada de Qualidade de Imagens",
    showlegend=False
)

fig_image_quality.show()

print("Visualização de análise de imagens criada")

Criando Visualizações Avançadas


Visualização de análise de imagens criada


In [19]:

fig_text_analysis = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribuição de Sentimento', 'Palavras por Texto',
                   'Legibilidade vs Complexidade', 'Idiomas Detectados'),
    specs=[[{"type": "pie"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)


sentiment_counts = text_results_df['sentiment_label'].value_counts()
fig_text_analysis.add_trace(
    go.Pie(labels=sentiment_counts.index, values=sentiment_counts.values,
           name="Sentiment Distribution"),
    row=1, col=1
)


fig_text_analysis.add_trace(
    go.Histogram(x=text_results_df['word_count'], name='Word Count', nbinsx=8),
    row=1, col=2
)


fig_text_analysis.add_trace(
    go.Scatter(x=text_results_df['flesch_score'], y=text_results_df['complexity'],
              mode='markers', name='Readability vs Complexity',
              marker=dict(size=15, color=text_results_df['positive_score'],
                         colorscale='RdYlBu', showscale=True),
              text=text_results_df['text_id']),
    row=2, col=1
)


language_counts = text_results_df['detected_language'].value_counts()
fig_text_analysis.add_trace(
    go.Bar(x=language_counts.index, y=language_counts.values, name='Languages'),
    row=2, col=2
)

fig_text_analysis.update_layout(
    height=800,
    title_text="Análise Avançada de Texto e NLP",
    showlegend=False
)

fig_text_analysis.show()

print("Visualização de análise de texto criada")

Visualização de análise de texto criada


In [20]:

print("\nDashboard de Performance do Pipeline")

combined_stats = {
    'Images Processed': len(image_results_df),
    'Texts Processed': len(text_results_df),
    'Avg Image Quality': round(image_results_df['quality_score'].mean(), 1),
    'Positive Sentiment Rate': round((text_results_df['sentiment_label'] == 'POSITIVE').mean() * 100, 1),
    'Processing Success Rate': 100.0,  # Todos foram processados com sucesso
    'Cache Hit Rate': round((image_stats['cache_entries'] / image_stats['total_processed']) * 100, 1)
}


fig_dashboard = go.Figure()


metrics = list(combined_stats.keys())
values = list(combined_stats.values())

fig_dashboard.add_trace(go.Bar(
    x=metrics,
    y=values,
    marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'],
    text=values,
    textposition='auto',
))

fig_dashboard.update_layout(
    title="Pipeline Performance Dashboard",
    xaxis_title="Metrics",
    yaxis_title="Values",
    height=500,
    showlegend=False
)

fig_dashboard.update_xaxes(tickangle=45)
fig_dashboard.show()

print("Dashboard de performance criado")


Dashboard de Performance do Pipeline


Dashboard de performance criado


## Resultados e Export de Dados

Compilação final dos resultados e demonstração de capacidades de export.

In [21]:
print("RESUMO FINAL DA DEMONSTRAÇÃO")
print("=" * 50)


print("\nResultados Alcançados:")
print(f"Imagens analisadas: {len(image_results_df)}")
print(f"Textos processados: {len(text_results_df)}")
print(f"Taxa de sucesso: 100%")
print(f"Visualizações criadas: 3 dashboards interativos")
print(f"Cache utilizado: {image_stats['cache_entries']} entradas")


print("\n Insights Extraídos:")
print(f"Qualidade média das imagens: {image_results_df['quality_score'].mean():.1f}/100")
print(f"Resolução média: {(image_results_df['width'] * image_results_df['height']).mean()/1000000:.1f}MP")
print(f"Taxa de sentimento positivo: {(text_results_df['sentiment_label'] == 'POSITIVE').mean()*100:.1f}%")
print(f"Legibilidade média: {text_results_df['flesch_score'].mean():.1f} (Flesch Score)")
print(f"Idioma principal detectado: {text_results_df['detected_language'].mode()[0]}")


print("\n Capacidades Técnicas Demonstradas:")
capabilities = [
    "✓ Pipeline de dados end-to-end",
    "✓ Processamento em lote eficiente",
    "✓ Cache inteligente para otimização",
    "✓ Logging estruturado e monitoramento",
    "✓ Tratamento robusto de erros",
    "✓ Análise multimodal (imagem + texto)",
    "✓ Visualizações interativas avançadas",
    "✓ Métricas de qualidade e performance",
    "✓ Código modular e reutilizável",
    "✓ Integração com bibliotecas modernas"
]

for capability in capabilities:
    print(f"   {capability}")

print("\n" + "=" * 50)
print("DEMONSTRAÇÃO CONCLUÍDA COM SUCESSO!")
print("=" * 50)

RESUMO FINAL DA DEMONSTRAÇÃO

Resultados Alcançados:
Imagens analisadas: 5
Textos processados: 5
Taxa de sucesso: 100%
Visualizações criadas: 3 dashboards interativos
Cache utilizado: 5 entradas

 Insights Extraídos:
Qualidade média das imagens: 99.4/100
Resolução média: 0.5MP
Taxa de sentimento positivo: 80.0%
Legibilidade média: 0.0 (Flesch Score)
Idioma principal detectado: english

 Capacidades Técnicas Demonstradas:
   ✓ Pipeline de dados end-to-end
   ✓ Processamento em lote eficiente
   ✓ Cache inteligente para otimização
   ✓ Logging estruturado e monitoramento
   ✓ Tratamento robusto de erros
   ✓ Análise multimodal (imagem + texto)
   ✓ Visualizações interativas avançadas
   ✓ Métricas de qualidade e performance
   ✓ Código modular e reutilizável
   ✓ Integração com bibliotecas modernas

DEMONSTRAÇÃO CONCLUÍDA COM SUCESSO!


In [22]:

print("Exportando resultados...")


image_results_df.to_csv('image_analysis_results.csv', index=False)
text_results_df.to_csv('text_analysis_results.csv', index=False)


final_report = {
    'execution_timestamp': datetime.now().isoformat(),
    'pipeline_summary': {
        'images_processed': len(image_results_df),
        'texts_processed': len(text_results_df),
        'success_rate': 100.0
    },
    'image_analysis_summary': {
        'avg_quality_score': float(image_results_df['quality_score'].mean()),
        'avg_resolution_mp': float((image_results_df['width'] * image_results_df['height']).mean() / 1000000),
        'quality_distribution': image_results_df['quality_score'].describe().to_dict()
    },
    'text_analysis_summary': {
        'positive_sentiment_rate': float((text_results_df['sentiment_label'] == 'POSITIVE').mean()),
        'avg_readability': float(text_results_df['flesch_score'].mean()),
        'avg_complexity': float(text_results_df['complexity'].mean()),
        'language_distribution': text_results_df['detected_language'].value_counts().to_dict()
    },
    'performance_metrics': {
        'image_processing_stats': image_stats,
        'text_processing_stats': text_stats
    }
}


with open('data_engineering_.json', 'w') as f:
    json.dump(final_report, f, indent=2, default=str)

print("Exportação concluída!")
print("\n Arquivos gerados:")
print("   • image_analysis_results.csv")
print("   • text_analysis_results.csv")
print("   • data_engineering_.json")

print("\n Pronto para apresentação e portfolio!")

Exportando resultados...
Exportação concluída!

 Arquivos gerados:
   • image_analysis_results.csv
   • text_analysis_results.csv
   • data_engineering_.json

 Pronto para apresentação e portfolio!
